# 4. Inference

In [1]:
import sys, os
sys.path.append("../../..")

import pathSetting
import pandas as pd

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
from Clust.clust.ML.common.common import p2_dataSelection as p2
from Clust.clust.ML.common.common import p4_testing as p4
from Clust.clust.ML.forecasting import app

import torch
#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu is available.


## 4-1. (just for test) Data Preparation 

In [2]:
LearningMode = "test"
cleanParamList =["Clean", "NoClean"]
datasetNameList=['Hs1SwineFarmWithWeatherTime', 'gunwiStrawberryWithWeatherTime', 'strawberryOpenTime']

DataMeta = p1.read_json_data(pathSetting.DataMetaPath)
print()
### 1-1. Change Variable##########################
datasetName = datasetNameList[0]
cleanMode =cleanParamList[1]
##################################################
dataName ="test"+cleanMode+'_'+datasetName
dataSaveMode = DataMeta[dataName]["integrationInfo"]["DataSaveMode"]
print(dataName)

# 2
ModelMeta =p1.read_json_data(pathSetting.trainModelMetaFilePath)
model_method ="gru"
modelName = "train"+cleanMode+'_'+datasetName+'_'+model_method

data = p2.get_saved_integrated_data(dataSaveMode, dataName, pathSetting.dataFolderPath, db_client = pathSetting.db_client)


testNoClean_Hs1SwineFarmWithWeatherTime


In [3]:
from Clust.clust.ingestion.mongo.mongo_client import MongoClient
from Clust.setting import influx_setting_KETI as ins

mongo_client = MongoClient(ins.CLUSTMetaInfo2)

db_name = 'model'
collection_name = 'meta'

all_model_meta = mongo_client.get_all_document(db_name, collection_name)

In [4]:
search = {'modelName': 'trainNoClean_Hs1SwineFarmWithWeatherTime_gru'}
model_meta= mongo_client.get_document_by_json(db_name, collection_name, search)
model_meta = model_meta[0]

In [5]:
model_meta

{'trainDataInfo': {'startTime': '2021-02-01 00:00:00',
  'endTime': '2021-03-10 00:00:00',
  'dataInfo': [['farm_swine_air', 'HS2'],
   ['weather_outdoor_keti_clean', 'sangju'],
   ['life_additional_Info', 'trigonometicInfoByHours']],
  'processParam': {'refine_param': {'removeDuplication': {'flag': False},
    'staticFrequency': {'flag': False, 'frequency': None}},
   'outlier_param': {'certainErrorToNaN': {'flag': False},
    'unCertainErrorToNaN': {'flag': False, 'param': {}}},
   'imputation_param': {'flag': False,
    'imputation_method': [],
    'totalNonNanRatio': 80}},
  'integration_freq_sec': 300,
  'cleanParam': 'NoClean',
  'DataSaveMode': 'CSV'},
 'modelName': 'trainNoClean_Hs1SwineFarmWithWeatherTime_gru',
 'featureList': ['Temperature', 'out_temp', 'sin_hour'],
 'trainDataType': 'timeseries',
 'modelPurpose': 'forecasting',
 'model_method': 'gru',
 'modelTags': ['farm', 'HS2', 'prediction'],
 'cleanTrainDataParam': 'NoClean',
 'NaNProcessingParam': {'feature_cycle': 'Day

In [6]:
scaler_file_path = model_meta['files']['scalerFile']["filePath"]
model_file_path = model_meta['files']['modelFile']["filePath"]

feature_list = model_meta["featureList"]
target_col = model_meta['transformParameter']["target_col"]
scaler_param = model_meta["scalerParam"]
past_step = model_meta['transformParameter']['past_step']

feature_data = data[feature_list]
step_data = feature_data[-past_step:][feature_list].values
df_data = pd.DataFrame(step_data, columns = feature_list)

input_data, scaler = p4.get_scaled_test_data(df_data[feature_list], scaler_file_path, scaler_param)

In [7]:
from Clust.clust.ML.regression_YK.inference_forecast import ForecastInference as FI

fi = FI()
fi.set_param(model_meta)
fi.set_model(model_method, model_file_path)
fi.set_data(input_data)
preds = fi.inference()


Start inference

** Dimension of result for inference dataset = (1,)


In [5]:
# prediction_result = app.get_inference_result(data, model_meta)


Start testing data

======================1111111111111111111111111111111111=================================
[0.24773747]


In [10]:
if scaler_param =='scale':
    base_df_for_inverse= pd.DataFrame(columns=feature_list, index=range(len(preds)))
    base_df_for_inverse[target_col] = preds
    inverse_result = pd.DataFrame(scaler.inverse_transform(base_df_for_inverse), columns=feature_list, index=base_df_for_inverse.index)
    target_data = inverse_result[target_col]
    prediction_result = pd.DataFrame(data={target_col: target_data}, index=range(len(preds)))
        
else:
    prediction_result = pd.DataFrame(data={target_col: preds}, index=range(len(preds)))

In [12]:
prediction_result

,Temperature
0,22.430008
